In [ ]:
import os
import numpy as np
import pandas as pd


# =========================================================
# 0) 공통: CSV 로드 유틸
# =========================================================
def read_csv_fallback(path: str) -> pd.DataFrame:
    try:
        return pd.read_csv(path, encoding="cp949")
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding="utf-8-sig")


# =========================================================
# 1) 역사운영 현황 로드/전처리
# =========================================================
def load_and_prepare_station_facility(csv_path: str) -> pd.DataFrame:
    df = read_csv_fallback(csv_path)

    # 컬럼 표준화
    df = df.rename(columns={
        "호선": "line_raw",
        "역명": "역명",
        "면적": "station_area",
        "승강장유형": "platform_type",
        "환승노선": "transfer_lines",
    })

    # 호선 숫자 추출 (예: '1호선' -> 1)
    df["line"] = (
        df["line_raw"].astype(str)
        .str.extract(r"(\d+)", expand=False)
    )
    df["line"] = pd.to_numeric(df["line"], errors="coerce").astype("Int64")

    # 면적 숫자화 (쉼표 제거 대비)
    df["station_area"] = (
        df["station_area"].astype(str)
        .str.replace(",", "", regex=False)
    )
    df["station_area"] = pd.to_numeric(df["station_area"], errors="coerce")

    # 승강장유형 더미
    df["platform_type"] = df["platform_type"].astype(str).str.strip()
    df["is_island_platform"] = (df["platform_type"] == "섬식").astype("Int8")
    df["is_side_platform"] = (df["platform_type"] == "상대식").astype("Int8")

    # 환승노선 개수
    df["transfer_lines"] = df["transfer_lines"].fillna("").astype(str)
    df["num_transfer_lines"] = df["transfer_lines"].apply(
        lambda x: 0 if x.strip() == "" else len([t for t in x.split(",") if t.strip()])
    ).astype("Int16")

    # 키 정리
    df["역명"] = df["역명"].astype(str).str.strip()

    # feature 테이블 반환
    feat = (
        df[["line", "역명", "station_area", "is_island_platform", "is_side_platform", "num_transfer_lines"]]
        .dropna(subset=["line", "역명"])
        .drop_duplicates(subset=["line", "역명"])
        .reset_index(drop=True)
    )
    return feat


# =========================================================
# 2) df_final에 붙이기 (line, 역명 기준)
# =========================================================
def attach_station_facility_to_df_final(
    df_final: pd.DataFrame,
    facility_feat: pd.DataFrame,
    line_col: str = "line",
    name_col: str = "역명",
) -> pd.DataFrame:
    out = df_final.copy()

    # 키 정리
    out[line_col] = pd.to_numeric(out[line_col], errors="coerce").astype("Int64")
    out[name_col] = out[name_col].astype(str).str.strip()

    out = out.merge(
        facility_feat,
        left_on=[line_col, name_col],
        right_on=["line", "역명"],
        how="left",
        suffixes=("", "_fac"),
    )

    # 오른쪽 키 컬럼 정리
    out = out.drop(columns=[c for c in ["line_fac", "역명_fac"] if c in out.columns], errors="ignore")

    # 결측 처리: 환승노선/플랫폼 여부는 0
    for c in ["is_island_platform", "is_side_platform", "num_transfer_lines"]:
        if c in out.columns:
            out[c] = out[c].fillna(0).astype("Int16")

    if "station_area" in out.columns:
        out["station_area"] = pd.to_numeric(out["station_area"], errors="coerce").astype("float")

    return out


# =========================================================
# 3) 승하차 파생변수
# =========================================================
def add_board_alight_features(
    df: pd.DataFrame,
    ride_col: str = "승차",
    alight_col: str = "하차",
    area_col: str = "station_area",
) -> pd.DataFrame:
    out = df.copy()

    out[ride_col] = pd.to_numeric(out[ride_col], errors="coerce")
    out[alight_col] = pd.to_numeric(out[alight_col], errors="coerce")
    if area_col in out.columns:
        out[area_col] = pd.to_numeric(out[area_col], errors="coerce")

    out["board_alight_sum"] = out[ride_col].fillna(0) + out[alight_col].fillna(0)
    out["board_alight_diff"] = out[ride_col].fillna(0) - out[alight_col].fillna(0)

    # 면적 대비 혼잡도: 면적 없거나 0이면 NaN
    if area_col in out.columns:
        area = out[area_col].where(out[area_col] > 0, np.nan)
        out["platform_congestion"] = out["board_alight_sum"] / area
    else:
        out["platform_congestion"] = np.nan

    return out


# =========================================================
# 4) 컬럼 정리 유틸
# =========================================================
def clean_station_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 역명 통합
    if "역명_x" in out.columns:
        out["역명"] = out["역명_x"]
    elif "역명_y" in out.columns:
        out["역명"] = out["역명_y"]

    drop_cols = [c for c in ["역명_x", "역명_y"] if c in out.columns]
    out = out.drop(columns=drop_cols, errors="ignore")

    # station_cd 정리
    if "station_cd" in out.columns:
        out["station_cd"] = pd.to_numeric(out["station_cd"], errors="coerce").astype("Int64")

    if "station_cd_map" in out.columns:
        out = out.drop(columns=["station_cd_map"], errors="ignore")

    return out


def drop_intermediate_columns(df: pd.DataFrame) -> pd.DataFrame:
    drop_cols = ["dt", "apply_dt_x", "apply_dt_y", "release_dt", "month_str"]
    drop_cols = [c for c in drop_cols if c in df.columns]
    return df.drop(columns=drop_cols, errors="ignore")


def clean_cpi_columns(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 최종 CPI 변수만 유지
    keep = ["cpi_yoy_actual_filled"]
    drop = [c for c in out.columns if c.startswith("cpi_yoy_") and c not in keep]
    out = out.drop(columns=drop, errors="ignore")

    if "cpi_yoy_actual_filled" in out.columns:
        out = out.rename(columns={"cpi_yoy_actual_filled": "cpi_yoy"})

    return out


def reorder_columns(df: pd.DataFrame) -> pd.DataFrame:
    priority = [
        "날짜", "line", "station_number", "station_cd", "역명",
        "승객유형", "승차", "하차",
        "hour", "tm", "ym",
        "up_trip_cnt", "down_trip_cnt",
        "up_mean_headway", "down_mean_headway",
    ]
    exist_priority = [c for c in priority if c in df.columns]
    rest = [c for c in df.columns if c not in exist_priority]
    return df[exist_priority + rest]


# =========================================================
# 5) 파이프라인 (df_final.csv에서 시작)
# =========================================================
def build_df_final_from_raw(
    df_final_path: str,
    facility_path: str,
) -> pd.DataFrame:
    df_final = read_csv_fallback(df_final_path)

    # 기본 키 정리
    if "line" in df_final.columns:
        df_final["line"] = pd.to_numeric(df_final["line"], errors="coerce").astype("Int64")

    # 시설 feature 준비 + merge
    facility_feat = load_and_prepare_station_facility(facility_path)
    df_final = attach_station_facility_to_df_final(df_final, facility_feat, line_col="line", name_col="역명")

    # 승하차 파생변수
    df_final = add_board_alight_features(df_final)

    # 컬럼 클린업/정리
    df_final = clean_station_columns(df_final)
    df_final = drop_intermediate_columns(df_final)
    df_final = clean_cpi_columns(df_final)
    df_final = reorder_columns(df_final)

    return df_final


# =========================================================
# 6) 실행부
# =========================================================
if __name__ == "__main__":
    DATA_DIR = "./data"
    DF_FINAL_PATH = os.path.join(DATA_DIR, "df_final.csv")
    FACILITY_PATH = os.path.join(DATA_DIR, "서울교통공사_역사운영 현황_20250310.csv")

    OUT_V2_PATH = os.path.join(DATA_DIR, "df_final_v2.csv")

    df_final = build_df_final_from_raw(
        df_final_path=DF_FINAL_PATH,
        facility_path=FACILITY_PATH,
    )

    # 전체 v2 저장
    df_final.to_csv(OUT_V2_PATH, index=False, encoding="utf-8-sig")


    # 간단 체크
    if "station_area" in df_final.columns:
        print("[CHECK] station_area missing rate:", df_final["station_area"].isna().mean())
    if "is_island_platform" in df_final.columns:
        print("[CHECK] is_island_platform missing rate:", df_final["is_island_platform"].isna().mean())
    if "num_transfer_lines" in df_final.columns:
        print("[CHECK] num_transfer_lines missing rate:", df_final["num_transfer_lines"].isna().mean())

    cols_to_show = [c for c in ["역명", "line", "station_area", "is_island_platform", "is_side_platform", "num_transfer_lines"] if c in df_final.columns]
    if cols_to_show:
        print(df_final[cols_to_show].drop_duplicates().head(20))

    print(f"[INFO] saved -> {OUT_V2_PATH}")
    print("[INFO] shape:", df_final.shape)
    print("[INFO] columns:", df_final.columns.tolist())

[CHECK] station_area missing rate: 0.0
[CHECK] is_island_platform missing rate: 0.0
[CHECK] num_transfer_lines missing rate: 0.0
         역명  line  station_area  is_island_platform  is_side_platform  \
0       서울역     1      10805.00                   1                 0   
1        혜화     4       6039.00                   0                 1   
2        합정     6      11229.45                   0                 1   
5      건대입구     2       6543.00                   0                 1   
9      종로3가     5       9188.01                   1                 0   
13       성수     2      10045.00                   0                 0   
14       선릉     2       7396.00                   0                 1   
16       역삼     2       8352.89                   0                 1   
19      여의도     5      12889.57                   0                 1   
22       신사     3       7159.00                   0                 1   
23       강남     2       6392.00                   0                 